<a href="https://colab.research.google.com/github/mashcodes10/yrikka-1A/blob/Yolo-predictions/Copy_of_Yrikka1A_Milestone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.6 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO("yolo11n.pt")

In [ ]:
from ultralytics import YOLO
import os, shutil, csv, json
from collections import defaultdict

# -----------------------------
# Step 1. Run YOLO Predictions
# -----------------------------
model = YOLO("yolo11n.pt")

results = model.predict(
    source="cleaned_dataset/images",
    save=True,
    conf=0.25,
    classes=[58, 56, 41, 75, 73]
)

# -----------------------------
# Step 2. Class Mapping
# -----------------------------
coco_to_canonical = {
    58: 0,  # potted plant
    56: 1,  # chair
    41: 2,  # cup
    75: 3,  # vase
    73: 4   # book
}
canonical_names = ["potted plant", "chair", "cup", "vase", "book"]

# -----------------------------
# Step 3. Load Ground Truth
# -----------------------------
with open("cleaned_dataset/cleaned_coco.json") as f:
    gt = json.load(f)

image_id_to_ann = defaultdict(list)
for ann in gt["annotations"]:
    image_id_to_ann[ann["image_id"]].append(ann)

image_id_to_name = {img["id"]: img["file_name"] for img in gt["images"]}

# -----------------------------
# Step 4. IOU Function
# -----------------------------
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    inter = interW * interH
    areaA = boxA[2] * boxA[3]
    areaB = boxB[2] * boxB[3]
    union = areaA + areaB - inter
    return inter / union if union > 0 else 0

# -----------------------------
# Step 5. Compare Predictions vs GT
# -----------------------------
misclassified_images = set()
report_rows = []
iou_threshold = 0.5

for result in results:
    file_name = os.path.basename(result.path)

    # Match GT image id
    image_id = None
    for img in gt["images"]:
        if img["file_name"] == file_name:
            image_id = img["id"]
            break
    if image_id is None:
        continue

    gt_anns = image_id_to_ann[image_id]
    gt_boxes = [(ann["bbox"], ann["category_id"]) for ann in gt_anns]

    # Predictions
    preds = []
    for box in result.boxes:
        coco_id = int(box.cls.cpu().numpy()[0])
        if coco_id in coco_to_canonical:
            pred_class = coco_to_canonical[coco_id]
            xyxy = box.xyxy.cpu().numpy()[0]
            w, h = xyxy[2] - xyxy[0], xyxy[3] - xyxy[1]
            preds.append(([float(xyxy[0]), float(xyxy[1]), float(w), float(h)], pred_class))

    matched_preds = set()

    # Check GT objects against predictions
    for gt_box, gt_cls in gt_boxes:
        found = False
        for i, (pred_box, pred_cls) in enumerate(preds):
            if i in matched_preds:
                continue
            if iou(gt_box, pred_box) >= iou_threshold:
                found = True
                matched_preds.add(i)
                if pred_cls != gt_cls:
                    misclassified_images.add(file_name)
                    report_rows.append([
                        file_name, "Wrong Class",
                        canonical_names[gt_cls], canonical_names[pred_cls],
                        round(iou(gt_box, pred_box), 3)
                    ])
                break
        if not found:
            misclassified_images.add(file_name)
            report_rows.append([
                file_name, "Missed Detection",
                canonical_names[gt_cls], "None", 0.0
            ])

    # Also check for extra predictions not matched to any GT
    for i, (pred_box, pred_cls) in enumerate(preds):
        if i not in matched_preds:
            misclassified_images.add(file_name)
            report_rows.append([
                file_name, "Missing Annotation",
                "None", canonical_names[pred_cls], 0.0
            ])

# -----------------------------
# Step 6. Save Outputs
# -----------------------------

# Misclassified COCO JSON
misclassified_gt = {
    "images": [img for img in gt["images"] if img["file_name"] in misclassified_images],
    "annotations": [ann for ann in gt["annotations"]
                    if image_id_to_name[ann["image_id"]] in misclassified_images],
    "categories": gt["categories"]
}
with open("misclassified_coco.json", "w") as f:
    json.dump(misclassified_gt, f, indent=2)

# Copy misclassified images
out_dir = "misclassified_images"
os.makedirs(out_dir, exist_ok=True)
for img in misclassified_gt["images"]:
    src = os.path.join("cleaned_dataset/images", img["file_name"])
    dst = os.path.join(out_dir, img["file_name"])
    if os.path.exists(src):
        shutil.copy2(src, dst)

# CSV Report
with open("misclassified_report.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image", "Reason", "GroundTruth", "Prediction", "IOU"])
    writer.writerows(report_rows)

print(f"✅ Saved {len(misclassified_gt['images'])} misclassified/missed images")
print("📁 misclassified_images/ folder created")
print("📄 misclassified_report.csv generated")



image 1/497 /content/cleaned_dataset/images/0067dc2e-4bb7-48a2-b9c5-9da631f86595_flux_1756081928.png: 640x640 1 potted plant, 1 vase, 383.2ms
image 2/497 /content/cleaned_dataset/images/007abd03-fa5e-43d8-af0a-fbed58e17821_flux_1756081085.png: 640x640 (no detections), 224.0ms
image 3/497 /content/cleaned_dataset/images/00f8644a-c4af-4097-a0bc-e8af706554ce_flux_1756082211.png: 640x640 1 vase, 192.5ms
image 4/497 /content/cleaned_dataset/images/02de1ef0-1130-473d-9d2a-425b231ebdd2_flux_1756080597.png: 640x640 2 chairs, 2 potted plants, 187.4ms
image 5/497 /content/cleaned_dataset/images/0305fc14-9c1f-4b9e-b08b-33d199bc022f_flux_1756081386.png: 640x640 1 chair, 191.1ms
image 6/497 /content/cleaned_dataset/images/0313d4d2-c1b6-411a-9d0d-e29714a0d27e_flux_1756081964.png: 640x640 2 chairs, 2 potted plants, 1 book, 189.8ms
image 7/497 /content/cleaned_dataset/images/03af0e86-1d0e-4470-ae4a-dc34f3129c16_flux_1756082373.png: 640x640 (no detections), 199.8ms
image 8/497 /content/cleaned_dataset

In [ ]:
import shutil
from google.colab import files

# Create a zip of the misclassified images folder
shutil.make_archive("misclassified_images", "zip", "misclassified_images")

# Download the zip
files.download("misclassified_images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>